In [27]:
import pandas as pd
import numpy as np
from scipy import stats
from collections import Counter




In [28]:
def load(path):
 data = pd.read_csv(path, delimiter=',')
 
 return data
data=load("DatasetExos.csv")

In [29]:

data['ID'] = data['ID'].astype('category')  
data['Exercise'] = data['Exercise'].astype('category')  
# data.rename(columns={'Label': 'Exercise', }, inplace=True)
data['Category'] = data['Category'].astype('category')  
data['Set'] = data['Set'].where(data['Set'] >= 0,0) 
data['Set'] = data['Set'].fillna(0)  
data['Set']= pd.to_numeric(data['Set'], errors='coerce') 

print(data.dtypes)

ep (ms)       object
Acc_x        float64
Acc_y        float64
Acc_z        float64
Gyro_x       float64
Gyro_y       float64
Gyro_z       float64
ID          category
Exercise    category
Category    category
Set          float64
dtype: object


In [30]:
data = data.iloc[:, 7:]

# Print the updated DataFrame
print(data)

     ID Exercise Category   Set
0     B    bench    heavy  30.0
1     B    bench    heavy  30.0
2     B    bench    heavy  30.0
3     B    bench    heavy  30.0
4     B    bench    heavy  30.0
...  ..      ...      ...   ...
9004  E      row   medium  40.0
9005  E      row   medium  40.0
9006  E      row   medium  40.0
9007  E      row   medium  40.0
9008  E      row   medium  40.0

[9009 rows x 4 columns]


In [31]:

df = data

# Step 1: Rename values in the 'Exercise' and 'Category' columns
df['Exercise'] = df['Exercise'].replace({'reste': 'rest', 'raw': 'row'})
df['Category'] = df['Category'].replace({'heav': 'heavy'})




C:\Users\belou\AppData\Local\Temp\ipykernel_16940\4002143544.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['Exercise'] = df['Exercise'].replace({'reste': 'rest', 'raw': 'row'})
C:\Users\belou\AppData\Local\Temp\ipykernel_16940\4002143544.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['Category'] = df['Category'].replace({'heav': 'heavy'})


In [32]:
df = df.apply(lambda x: x.fillna(x.mode()[0]) if x.mode().size > 0 else x)

# Verify if missing values are filled
print("Missing values per column after replacement:\n", df.isnull().sum())


Missing values per column after replacement:
 ID          0
Exercise    0
Category    0
Set         0
dtype: int64


In [33]:
unique_id_sum = df["ID"].unique()
print(f"Sum of unique IDs: {len(unique_id_sum)}")

# Get unique combinations of 'Exercise', 'Category', and 'Set'
unique_combinations = df[['Exercise', 'Category', 'Set']].drop_duplicates()

# Print the number of unique combinations
num_unique_combinations = len(unique_combinations)
print(f"Number of unique combinations: {num_unique_combinations}")




Sum of unique IDs: 5
Number of unique combinations: 100


In [34]:

transactions_df = (
    df.groupby("ID")[['Exercise', 'Category', 'Set']]
    .apply(lambda group: group.drop_duplicates().agg(tuple, axis=1).tolist())
    .reset_index(name="Items")
)
transactions_df.rename(columns={"ID": "Transaction"}, inplace=True)


# Print the resulting DataFrame to verify
print("Transactions DataFrame:")
print(transactions_df)
# Save the DataFrame to CSV
transactions_df.to_csv("DatasetExos_2.csv", index=False)

print("DatasetExos_2 created with transactional format.")
import ast

# Read the CSV
transactions_df = pd.read_csv("DatasetExos_2.csv")
print(transactions_df)

# # Convert the 'Items' column back to a list of tuples
# transactions_df['Items'] = transactions_df['Items'].apply(ast.literal_eval)
# print(transactions_df)


Transactions DataFrame:
  Transaction                                              Items
0           A  [(bench, heavy, 86.0), (bench, heavy, 5.0), (o...
1           B  [(bench, heavy, 30.0), (bench, medium, 0.0), (...
2           C  [(bench, heavy, 33.0), (bench, heavy, 72.0), (...
3           D  [(squat, medium, 2.0), (squat, medium, 0.0), (...
4           E  [(bench, heavy, 80.0), (bench, heavy, 7.0), (d...
DatasetExos_2 created with transactional format.
  Transaction                                              Items
0           A  [('bench', 'heavy', 86.0), ('bench', 'heavy', ...
1           B  [('bench', 'heavy', 30.0), ('bench', 'medium',...
2           C  [('bench', 'heavy', 33.0), ('bench', 'heavy', ...
3           D  [('squat', 'medium', 2.0), ('squat', 'medium',...
4           E  [('bench', 'heavy', 80.0), ('bench', 'heavy', ...


C:\Users\belou\AppData\Local\Temp\ipykernel_16940\3581592313.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("ID")[['Exercise', 'Category', 'Set']]


1- Write a function to generate the k-itemset candidates Ck
.
2- Write a function to calculate the support of the k-itemsets Ck
.
3- Write a function to generate the frequent k-itemsets Lk

In [ ]:
from itertools import combinations
import re

def candidates(itemlist, nocombinaison, data):
   
 
 items=set()

    # If itemlist is empty, extract unique items from data['Items']
 if not itemlist:
         for row in data['Items']:
            
            row= ast.literal_eval(row)

            for item in row:
               items.add(item)  # Add each item to the set
         candidates_list = [item for item in items]  # 
     
 else:
    # Generate all 2-combinations (change 2 to k as needed for k-itemsets)
    candidates_list=[]
    for comb in combinations(itemlist, 2):
        # Check if any subset of `comb` exists in `nocombinaison`
       if nocombinaison is None or not any(set(sub_comb).issubset(comb) for sub_comb in nocombinaison):
            candidates_list.append(comb)

 return candidates_list
# def candidates(itemlist, nocombinaison, data):
#     items = set()

#     if not itemlist:
#         # Extract unique items from dataset
#         for row in data["Items"]:
#             for item in ast.literal_eval(row):
#                 items.add(item)
#         return list(items)

#     # Generate k-combinations
#     return [comb for comb in combinations(itemlist, 2) if comb not in nocombinaison]


In [36]:
def load2(path):
 data = pd.read_csv(path, delimiter=',')
 
 return data
data2=load2("DatasetExos_2.csv")


2- Write a function to calculate the support of the k-itemsets Ck

In [37]:
# def calculate_support(dataset, candidates,k):
    
#     support_counts = {candidate: 0 for candidate in candidates}
#     total_transactions = len(dataset)
#     count =True 
#     for transaction in dataset["Items"]:
#         transaction_set = ast.literal_eval(transaction)
#         for candidate in candidates:
#            if(k!=1):
#             for  c in candidate:
                
                
#                 if c not in transaction_set:
#                     count=False
                
#             if count==  True:   
#                 support_counts[candidate] += 1
#             else:
#                count= True  

#            else:
#             if candidate in transaction_set:
#                 support_counts[candidate] += 1
#     for i in   candidates:
#         support_counts[i]=support_counts[i]/total_transactions

#     # Calculer le support en pourcentage
#     return support_counts

    
    
import ast

def calculate_support(dataset, candidates, k):
    # Initialiser le compteur de support pour chaque candidat
    support_counts = {candidate: 0 for candidate in candidates}
    total_transactions = len(dataset)

    # Parcourir chaque transaction
    for transaction in dataset["Items"]:
        transaction_set = set(ast.literal_eval(transaction))
        
        for candidate in candidates:
            # Vérification pour k > 1 (candidats comme tuples ou ensembles)
            if k != 1:
                if all(item in transaction_set for item in candidate):
                    support_counts[candidate] += 1
            # Vérification pour k == 1 (candidats comme éléments individuels)
            else:
                if candidate in transaction_set:
                    support_counts[candidate] += 1

    # Calculer le support en proportion
    support_percentages = {candidate: count / total_transactions for candidate, count in support_counts.items()}

    return support_percentages


In [38]:
def generate_frequent_itemsets(support, min_support):
  
    frequent_itemsets = []
    non_frequent_itemsets = []
    
    for itemset, supp in support.items():
        if supp >= min_support:
            frequent_itemsets.append(itemset)
        else:
            non_frequent_itemsets.append(itemset)
    
    return frequent_itemsets, non_frequent_itemsets

In [ ]:
# Minimum support and confidence
min_support = 0.1
L=[]
final=[]
nocomb=[]
b=True
i=0
while  b==True:
 C= candidates(L,nocomb,data2)
#  print(C)
 support_C2 = calculate_support(data2, C,i+1)
#  print(support_C2)
 L ,nocimb= generate_frequent_itemsets( support_C2, min_support)
#  print(L)
 if len(L)==0:
     b=False
 final.append(L)
 i+=1
print('final')
print(final)
# Step 1: Generate frequent itemsets (using earlier functions)


## association rules

In [ ]:
from itertools import chain, combinations

def generate_association_rules(Lk):
   
    rules = []
    i=0
    for it in Lk[1:]:
     
     
        # convert type to set 
      it = set(it)
      # print('it',it,len(it))
      for itemset in it :
        # print('itemset',itemset,len(itemset))
    
        itemset=set(itemset)
        for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    # print('and',antecedent,len(antecedent))
                    antecedent = set(antecedent)
                    consequent = itemset-antecedent
                    rules.append((antecedent, consequent))    
    return rules


In [ ]:
rules=generate_association_rules(final)
print(rules)

[({('row', 'medium', 4.0)}, {('row', 'heavy', 90.0)}), ({('row', 'heavy', 90.0)}, {('row', 'medium', 4.0)}), ({('row', 'heavy', 42.0)}, {('bench', 'heavy', 62.0)}), ({('bench', 'heavy', 62.0)}, {('row', 'heavy', 42.0)}), ({('dead', 'medium', 13.0)}, {('row', 'medium', 63.0)}), ({('row', 'medium', 63.0)}, {('dead', 'medium', 13.0)}), ({('dead', 'medium', 27.0)}, {('row', 'medium', 49.0)}), ({('row', 'medium', 49.0)}, {('dead', 'medium', 27.0)}), ({('row', 'medium', 36.0)}, {('bench', 'heavy', 72.0)}), ({('bench', 'heavy', 72.0)}, {('row', 'medium', 36.0)}), ({('dead', 'heavy', 13.0)}, {('row', 'medium', 32.0)}), ({('row', 'medium', 32.0)}, {('dead', 'heavy', 13.0)}), ({('bench', 'medium', 50.0)}, {('bench', 'medium', 34.0)}), ({('bench', 'medium', 34.0)}, {('bench', 'medium', 50.0)}), ({('row', 'heavy', 91.0)}, {('bench', 'medium', 31.0)}), ({('bench', 'medium', 31.0)}, {('row', 'heavy', 91.0)}), ({('dead', 'heavy', 13.0)}, {('rest', 'sitting', 89.0)}), ({('rest', 'sitting', 89.0)}, {('

In [ ]:
def calculate_confidence(rule, data2):
    
    A, B = rule
    AB = [(*A, *B)]
    support_AB = calculate_support(data2, AB,2)
   
    support_A =calculate_support(data2, A,len(A))
    # Make sure to extract the relevant numeric value from the dictionaries
    if isinstance(support_AB, dict):
        support_AB = list(support_AB.values())[0]  # Access the first value in the dictionary
    if isinstance(support_A, dict):
        support_A = list(support_A.values())[0]  # Access the first value in the dictionary

    
    return support_AB / support_A if support_A > 0 else 0


In [ ]:
for rule in rules:
#    if (calculate_confidence(rule, data2)!=1):
    print( rule ,':',calculate_confidence(rule, data2))

({('row', 'medium', 4.0)}, {('row', 'heavy', 90.0)}) : 1.0
({('row', 'heavy', 90.0)}, {('row', 'medium', 4.0)}) : 1.0
({('row', 'heavy', 42.0)}, {('bench', 'heavy', 62.0)}) : 1.0
({('bench', 'heavy', 62.0)}, {('row', 'heavy', 42.0)}) : 1.0
({('dead', 'medium', 13.0)}, {('row', 'medium', 63.0)}) : 1.0
({('row', 'medium', 63.0)}, {('dead', 'medium', 13.0)}) : 1.0
({('dead', 'medium', 27.0)}, {('row', 'medium', 49.0)}) : 1.0
({('row', 'medium', 49.0)}, {('dead', 'medium', 27.0)}) : 1.0
({('row', 'medium', 36.0)}, {('bench', 'heavy', 72.0)}) : 1.0
({('bench', 'heavy', 72.0)}, {('row', 'medium', 36.0)}) : 1.0
({('dead', 'heavy', 13.0)}, {('row', 'medium', 32.0)}) : 1.0
({('row', 'medium', 32.0)}, {('dead', 'heavy', 13.0)}) : 1.0
({('bench', 'medium', 50.0)}, {('bench', 'medium', 34.0)}) : 1.0
({('bench', 'medium', 34.0)}, {('bench', 'medium', 50.0)}) : 1.0
({('row', 'heavy', 91.0)}, {('bench', 'medium', 31.0)}) : 1.0
({('bench', 'medium', 31.0)}, {('row', 'heavy', 91.0)}) : 1.0
({('dead', '